In [1]:
import numpy as np
from pyscf import dft, gto

In [2]:
mol = gto.M(atom='O  0  0  0; H  0 1 0 ; H 0 0 1', basis='STO-3G')
auxmol = gto.M(atom='O  0  0  0; H  0 1 0 ; H 0 0 1', 
               basis=gto.basis.parse("""
                                        O    S
                                             4.0              1.0000000  
                                        O    S
                                             1.0              1.0000000
                                        O    S
                                             0.5              1.0000000
                                        """)
)

mf = dft.RKS(mol)
mf.xc = 'PBE'
mf.grids.level = 5
mf.kernel()

dm = mf.make_rdm1()

pmol = mol + auxmol

converged SCF energy = -75.2339187560294


In [3]:
mol.ao_labels()

['0 O 1s    ',
 '0 O 2s    ',
 '0 O 2px   ',
 '0 O 2py   ',
 '0 O 2pz   ',
 '1 H 1s    ',
 '2 H 1s    ']

In [4]:
auxmol.ao_labels()

['0 O 1s    ',
 '0 O 2s    ',
 '0 O 3s    ',
 '1 H 1s    ',
 '1 H 2s    ',
 '1 H 3s    ',
 '2 H 1s    ',
 '2 H 2s    ',
 '2 H 3s    ']

## Numerical integrals

$$ \zeta_\sigma(r)$$

In [5]:
ao_aux_eval = mf._numint.eval_ao(auxmol, mf.grids.coords)

In [6]:
ao_aux_eval.shape

(76490, 9)

$$ \rho(r) $$

In [14]:
ao_eval = mf._numint.eval_ao(mol, mf.grids.coords, deriv=1)

In [15]:
ao_eval.shape

(4, 76490, 7)

In [16]:
rho = mf._numint.eval_rho(mol, ao_eval, dm, xctype='GGA')

In [17]:
rho.shape

(4, 76490)

$$ c_\sigma = \int \rho(r) \zeta_\sigma(r) $$

In [115]:
np.einsum('is,i -> s', ao_aux_eval, rho*mf.grids.weights)

array([4.42162163, 3.00325556, 2.39249209, 0.2785002 , 0.523849  ,
       0.77510625, 0.2785002 , 0.523849  , 0.77510625])

## Analytic integrals

$$ \rho(r) = \rho_{\mu\nu} \psi_\mu(r) \psi_\nu(r) $$

$$ c_\sigma = \int \rho(r) \zeta_\sigma(r) = \rho_{\mu\nu} \int_r \psi_\mu(r) \psi_\nu(r) \zeta_\sigma(r) $$

$$ I_{\mu\nu\sigma} = \int_r \psi_\mu(r) \psi_\nu(r) \zeta_\sigma(r) $$

In [116]:
pmol = mol + auxmol

In [117]:
i3c = pmol.intor('int3c1e_sph', shls_slice=(0, mol.nbas, 0, mol.nbas, mol.nbas, mol.nbas + auxmol.nbas))

In [118]:
i3c.shape

(7, 7, 9)

$$ c_\sigma = \sum_{\mu,\nu} I_{\mu\nu\sigma}\rho_{\mu\nu} $$

In [119]:
projections = np.einsum('mn,mns -> s', dm, i3c)

In [120]:
projections

array([4.42162164, 3.00325556, 2.39249209, 0.2785002 , 0.523849  ,
       0.77510625, 0.2785002 , 0.523849  , 0.77510625])